In [1]:
import pandas as pd
import sys
import csv
from statistics import median

In [2]:
project_ytest_lib = {}

In [3]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [4]:
def get_best_confidence(data):
    
    #typecasting
    data['project_reqd_builds'] = pd.to_numeric(data['project_reqd_builds'], downcast="float", errors='coerce')   
    
    #setting parameters
    project_list = set(data['project'].tolist())
    batching_alg = set(data['algorithm'].tolist())
    batchsizes = [1,2,4,8,16]
    
    #result file
    result_file = open('with_delay_best_confidence.csv', 'w')
    res_headers = ['project', 'method', 'algorithm', 'batch_size', 'builds_saved', 'testall_size', 'confidence', 'median_delays', 'delay_list']
    res_writer = csv.writer(result_file)
    res_writer.writerow(res_headers)
    
    
    #get best confidences

    for p in project_list:
        
        p_name = p.split('/')[1]
        alg_scores = {}
        p_data = data[data['project'] == p]
        
        y_test = pd.read_csv('../data/25_1_travis_data/' + p)['tr_status'].tolist()
        y_test = output_values(y_test)
        
        for alg in batching_alg:

            alg_data = p_data[p_data['algorithm'] == alg]

            for b in batchsizes:

                if alg == 'BATCH4':
                    if b != 4:
                        continue

                if alg == 'BATCHSTOP4':
                    if b < 4:
                        continue
                        

                batch_data = alg_data[ alg_data['batch_size'] == b]
                
                total = batch_data['testall_size'].tolist()[0]
                reqd_list = batch_data['project_reqd_builds'].tolist()
                
                median_delays = batch_data['median_delay'].tolist()
                delay_lists = batch_data['delay_list'].tolist()
                
                min_distance = sys.maxsize

                best = [p_name, 'ssr', alg, b, 0, 0, 0, 0, []]
                best_i = 0

                for i in range(len(reqd_list)):
                    distance = ((reqd_list[i]**2)+(median_delays[i])**2)**0.5
                    if distance < min_distance:
                        best[4] = 100-reqd_list[i]
                        best[5] = total
                        best[6] = i+2
                        best[7] = median_delays[i]
                        best[8] = delay_lists[i]
                        best_i = i
                        min_distance = distance

                res_writer.writerow(best)
    

In [5]:
data = pd.read_csv('all_combined_results.csv')

In [10]:
data

,Unnamed: 0,version,project,algorithm,batch_size,confidence,project_reqd_builds,project_missed_builds,project_saved_builds,sbs_delays,testall_size,total_batch_delay,batch_delays,ci
0,76817,1,floragunncom-search-guard/floragunncom-search-...,BATCHBISECT,1,2,55.629505,0.110926,44.370494,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3606,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."
1,76818,1,floragunncom-search-guard/floragunncom-search-...,BATCHBISECT,1,3,39.462009,2.468109,60.537992,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ...",3606,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, ..."
2,76819,1,floragunncom-search-guard/floragunncom-search-...,BATCHBISECT,1,4,33.361065,0.388242,66.638935,"[0, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, ...",3606,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, ..."
3,76820,1,floragunncom-search-guard/floragunncom-search-...,BATCHBISECT,1,5,26.067665,4.104271,73.932335,"[0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 1, 0, ...",3606,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ..."
4,76821,1,floragunncom-search-guard/floragunncom-search-...,BATCHBISECT,1,6,22.878536,4.215197,77.121464,"[0, 0, 0, 0, 0, 1, 2, 3, 0, 1, 2, 0, 0, 0, 0, ...",3606,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13086,106699,1,ngageoint-geowave/ngageoint-geowave.csv,BATCHSTOP4,16,16,40.012783,3.020134,24.496644,"[7, 8, 9, 10, 11, 12, 0, 0, 0, 0, 0, 0, 0, 0, ...",6258,3600.0,"[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13087,106700,1,ngageoint-geowave/ngageoint-geowave.csv,BATCHSTOP4,16,17,35.266857,4.202621,27.372963,"[8, 9, 10, 11, 12, 13, 0, 0, 0, 0, 0, 0, 0, 0,...",6258,3600.0,"[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13088,106701,1,ngageoint-geowave/ngageoint-geowave.csv,BATCHSTOP4,16,18,38.462769,3.835091,27.085331,"[9, 10, 11, 12, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0...",6258,3486.0,"[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13089,106702,1,ngageoint-geowave/ngageoint-geowave.csv,BATCHSTOP4,16,19,36.097794,4.426334,28.331735,"[10, 11, 12, 13, 14, 15, 0, 0, 0, 0, 0, 0, 0, ...",6258,3480.0,"[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [7]:
data = data[data['project'] != 'BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv']
data = data[data['project'] != 'RS485-LogisticsPipes/RS485-LogisticsPipes.csv']
data = data[data['project'] != 'datastax-java-driver/datastax-java-driver.csv']

In [8]:
len(data)

12350

In [9]:
get_best_confidence(data)

KeyError: 'median_delay'

92

In [24]:
result_file = open('sbs_delay_lists.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'median_sbs_delays', 'median_batch_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

86

In [25]:
lines = []

In [26]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_sbs_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            ssr_delays = batch_data['project_delays'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = ssr_delays[i]
                    #best[5] = median(ssr_delays[i])
                    best[5] = i+2
                    best_i = i
                    min_distance = distance

            lines.append(best)
            if alg == 'BATCH4':
                print(best)

['cloudify.csv', 'ssr', 'BATCH4', 4, '[]', 189]


In [27]:
ssr_delay = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'sbs_delays', 'confidence'])

In [28]:
ssr_delay.to_csv('sbs_delay_list.csv')